In [1]:
import random
import re
import pandas as pd
import numpy as np
import json
import requests
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

originating_restaurant = None

# Upload JSON as Pandas Tables

In [2]:
sandwiches_table = pd.read_json("https://github.com/IsaacFigNewton/SandoMatch/raw/refs/heads/main/sandwich-dataset/db-tables/sandwiches.json")
ingredients_table = pd.read_json("https://github.com/IsaacFigNewton/SandoMatch/raw/refs/heads/main/sandwich-dataset/db-tables/restaurant_ingredients.json")

In [3]:
# basically reset the indices
sandwiches_table.drop(columns=["_id", "review_count"], inplace=True)

# reset the index
ingredients_table.set_index("_id", inplace=True)

In [4]:
sandwiches_table.head()

,cuisine,restaurant,ingredients,cost,calories,rating,dietary_tags,reviews,name
0,None,None,"{'breads': {}, 'meats': {'egg': ['egg']}, 'che...",2.8,343,4,[vegetarian],[],Egg Sandwich
1,None,None,"{'breads': {}, 'meats': {'egg': ['large egg'],...",4.4,503,4,[],[],Egg Sandwich
2,None,None,"{'breads': {}, 'meats': {}, 'cheeses': {'ricot...",3.2,175,4,[vegetarian],[],Ricotta Sandwich
3,cajun_creole,None,"{'breads': {'baguette': ['baguette']}, 'meats'...",9.8,635,5,[],[],Cajun Creole Shrimp Sandwich
4,spanish,None,"{'breads': {}, 'meats': {'salami': ['hard sala...",4.8,295,5,[],[],Spanish Salami Sandwich


In [5]:
ingredients_table.head()

,breads,meats,cheeses,vegetables,condiments,spices
_id,,,,,,
default,"{'baguette': {'cost': 1.5, 'calories': 250}, '...","{'beef': {'cost': 3.0, 'calories': 250}, 'chic...","{'cheddar': {'cost': 1.0, 'calories': 110}, 'm...","{'lettuce': {'cost': 0.5, 'calories': 10}, 'to...","{'mayonnaise': {'cost': 1.0, 'calories': 100},...","{'salt': {'cost': 0.1, 'calories': 0}, 'black ..."
mr_pickles,"{'dutch crunch': {'cost': 0.0, 'calories': Non...","{'turkey breast': {'cost': 13.99, 'calories': ...","{'swiss': {'cost': 0.0, 'calories': None}, 'ch...","{'red peppers': {'cost': 0.0, 'calories': None...","{'mayonnaise': {'cost': 0.0, 'calories': None}...","{'salt': {'cost': 0.0, 'calories': None}, 'pep..."
subway,"{'white': {'cost': 4.0, 'calories': '195'}, '9...","{'black forest ham': {'cost': 3.75, 'calories'...","{'smoked cheddar': {'cost': 3.75, 'calories': ...","{'avocado': {'cost': 0.0, 'calories': '75'}, '...","{'caesar dressing': {'cost': 0.0, 'calories': ...","{'salt': {'cost': 0.0, 'calories': None}, 'pep..."


# Helper Functions

In [6]:
# check if the ingredients are the same between 2 sandwiches
def check_subset(dict1, dict2):
  # if they're equal, regardless of type,
  #   they must be equal and thus each is a subset of the other
  if dict1 == dict2:
    return True

  for subdict1_key, subdict1 in dict1.items():
    # if there's an entry in dict1 not in dict2, it can't be a subset
    if subdict1_key not in dict2.keys():
      return False

    else:
      # if the entry in dict1 is NOT a subset of the respective entry in dict2
      if not check_subset(subdict1, dict2[subdict1_key]):
        return False

  # if all the entries in dict1 were indeed subsets of dict2 entries
  return True

In [7]:
def loop_through_ingredient_lists(sandwich_ingredients,
                                  restaurant_ingredients,
                                  checking_availability=False,
                                  estimate_type="cost"):
  if not checking_availability:
    estimate = 0

  # loop through the sandwich ingredients
  for category_name, category in sandwich_ingredients.items():

    available_ingredients = restaurant_ingredients[category_name].keys()
    for ingredient_name, ingredient in category.items():


      # if the broader ingredient type isn't available at a restaurant
      #   (ie no sando analogs exist)
      #   and availability is being checked
      if not ingredient_name in available_ingredients and checking_availability:
          return False

      # if the ingredient is available
      #   and estimates are being found
      if ingredient_name in available_ingredients and not checking_availability:
        try:
          # estimate the sandwich's associated cost or calorie count at a restaurant
          estimate += restaurant_ingredients[category_name][ingredient_name][estimate_type]
        except Exception as e:
          print(f"Error getting {ingredient_name} estimate: {e}")

  if checking_availability:
    return True
  else:
    return estimate

In [8]:
# add restaurant, cost, calorie info from one sando to another
def add_restaurant_data(base_sando, sandwich):
  decimal_places = 2

  base_sando["restaurants"].append(sandwich["restaurant"])
  base_sando["costs"].append(round(float(sandwich["cost"]), 2+decimal_places))
  base_sando["calories2"].append(sandwich["calories"])

# Modify the Tables

Change restaurant name to restaurant list field
- Check sando availability against ingredient lists

Change cost to cost list field
- 1 cost per restaurant
- Default cost estimate if no restaurant

Reviews list field: Points to review objects

## Clean the sandwiches_table

In [9]:
# Clean the sandwiches table
sandwiches_table["restaurant"] = sandwiches_table["restaurant"].astype(str)
sandwiches_table["cuisine"] = sandwiches_table["cuisine"].astype(str)
sandwiches_table.replace({"None": np.nan,
                          "nan": np.nan},
                         inplace=True)
sandwiches_table["cost"] = sandwiches_table["cost"].astype(float)
sandwiches_table["calories"] = sandwiches_table["calories"].astype(int)
sandwiches_table[["cost", "calories"]].fillna(0, inplace=True)
sandwiches_table["restaurant"].fillna("default", inplace=True)
sandwiches_table.dtypes

<ipython-input-9-e3cc4683196f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sandwiches_table[["cost", "calories"]].fillna(0, inplace=True)
<ipython-input-9-e3cc4683196f>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sandwiches_table["restaurant"].fillna("default", inplace=True)


,0
cuisine,object
restaurant,object
ingredients,object
cost,float64
calories,int64
rating,int64
dietary_tags,object
reviews,object
name,object


In [10]:
sandwiches_table.head()

,cuisine,restaurant,ingredients,cost,calories,rating,dietary_tags,reviews,name
0,NaN,default,"{'breads': {}, 'meats': {'egg': ['egg']}, 'che...",2.8,343,4,[vegetarian],[],Egg Sandwich
1,NaN,default,"{'breads': {}, 'meats': {'egg': ['large egg'],...",4.4,503,4,[],[],Egg Sandwich
2,NaN,default,"{'breads': {}, 'meats': {}, 'cheeses': {'ricot...",3.2,175,4,[vegetarian],[],Ricotta Sandwich
3,cajun_creole,default,"{'breads': {'baguette': ['baguette']}, 'meats'...",9.8,635,5,[],[],Cajun Creole Shrimp Sandwich
4,spanish,default,"{'breads': {}, 'meats': {'salami': ['hard sala...",4.8,295,5,[],[],Spanish Salami Sandwich


## Type conversion for ingredients table

In [11]:
def type_conversion(restaurant):
  for category_name, category in restaurant.items():

    # print(f"\tCategory: {category_name}")
    for ingredient_name, ingredient in category.items():

      # print(f"\t\tIngredient: {ingredient_name}")
      cost = ingredient["cost"]
      calories = ingredient["calories"]

      # handle cases of the value being "null"
      try:
        cost = float(cost)
      except Exception as e:
        cost = 0

      try:
        calories = int(calories)
      except Exception as e:
        calories = 0

      restaurant[category_name][ingredient_name]["cost"] = cost
      restaurant[category_name][ingredient_name]["calories"] = calories

In [12]:
# Type conversion for ingredients with null (or otherwise bad) values
ingredients_table.apply(type_conversion)
ingredients_table.dtypes

,0
breads,object
meats,object
cheeses,object
vegetables,object
condiments,object
spices,object


## Build the new sandwich list

In [13]:
sandwiches_table["restaurants"] = [[] for _ in range(len(sandwiches_table))]
sandwiches_table["costs"] = [[] for _ in range(len(sandwiches_table))]
sandwiches_table["calories2"] = [[] for _ in range(len(sandwiches_table))]

In [14]:
sandwiches_table.head()

,cuisine,restaurant,ingredients,cost,calories,rating,dietary_tags,reviews,name,restaurants,costs,calories2
0,NaN,default,"{'breads': {}, 'meats': {'egg': ['egg']}, 'che...",2.8,343,4,[vegetarian],[],Egg Sandwich,[],[],[]
1,NaN,default,"{'breads': {}, 'meats': {'egg': ['large egg'],...",4.4,503,4,[],[],Egg Sandwich,[],[],[]
2,NaN,default,"{'breads': {}, 'meats': {}, 'cheeses': {'ricot...",3.2,175,4,[vegetarian],[],Ricotta Sandwich,[],[],[]
3,cajun_creole,default,"{'breads': {'baguette': ['baguette']}, 'meats'...",9.8,635,5,[],[],Cajun Creole Shrimp Sandwich,[],[],[]
4,spanish,default,"{'breads': {}, 'meats': {'salami': ['hard sala...",4.8,295,5,[],[],Spanish Salami Sandwich,[],[],[]


### Handle duplicate sandwiches

In [15]:
sandwiches_table.shape[0]

196

In [16]:
# convert dictionaries to JSON strings for hashing
sandwiches_table["ingredients"] = sandwiches_table["ingredients"].apply(json.dumps)

In [17]:
# group the sandwiches
grouped_sandwiches = sandwiches_table.groupby("ingredients")

updated_rows = []
for name, group in grouped_sandwiches:
    # copy the first entry
    first_row = group.iloc[0].copy()

    # if 2 sandwiches are in the same group, but have different restaurants, costs, and/or calories associated with them
    if len(group) > 1:
        for _, row in group.iterrows():
            # then add the restaurant, cost, calorie info from every variation
            #   to the associated list fields in the first entry
            add_restaurant_data(first_row, row)
        updated_rows.append(first_row)

    # otherwise, if there's only 1 unique sando, add it to the updated dataframe
    else:
        add_restaurant_data(group.iloc[0], first_row)
        updated_rows.append(group.iloc[0])

# reconstruct the DataFrame
sandwiches_table = pd.DataFrame(updated_rows)

In [18]:
# convert the JSON strings back to dictionaries
sandwiches_table["ingredients"] = sandwiches_table["ingredients"].apply(json.loads)

In [19]:
sandwiches_table.shape[0]

190

In [20]:
sandwiches_table.head()

,cuisine,restaurant,ingredients,cost,calories,rating,dietary_tags,reviews,name,restaurants,costs,calories2
165,NaN,subway,{'breads': {'artisan italian bread': ['artisan...,6.99,500,5,[],[],Pickleball Club,[subway],[6.99],[500]
163,NaN,subway,{'breads': {'artisan italian bread': ['artisan...,10.99,590,1,[],[],The Monster,[subway],[10.99],[590]
32,jewish,default,"{'breads': {'bagel': ['bagel']}, 'meats': {}, ...",2.70,345,2,[vegetarian],[],Jewish Cream Cheese Sandwich,[default],[2.7],[345]
87,french,default,"{'breads': {'baguette': ['baguette']}, 'meats'...",5.00,505,1,[],[],French Beef Sandwich,[default],[5.0],[505]
57,british,default,"{'breads': {'baguette': ['baguette']}, 'meats'...",6.40,640,5,[],[],British Beef Sandwich,[default],[6.4],[640]


### Create restaurant-specific sandwich cost, calorie estimates, where applicable

In [21]:
def update_sandwich(sandwich):
  for restaurant_id, restaurant in ingredients_table.iterrows():

    # if the sandwich entry is not already associated with a restaurant,
    if not restaurant_id in sandwich["restaurants"]:

      # if a sandwich can be made at a given restaurant
      # check if all the ingredients in a sandwich are available at a given restaurant
      if loop_through_ingredient_lists(sandwich["ingredients"],
                              restaurant,
                              checking_availability=True):

        cost_estimate = round(float(loop_through_ingredient_lists(sandwich["ingredients"],
                              restaurant,
                              checking_availability=False,
                              estimate_type="cost")))

        # account for potential bread or meat mismatches for subway
        if cost_estimate < 5:
          print(f"Possibly undervalued for \t'{sandwich['name']}' for \t{restaurant_id}, adding $3.75.")
          cost_estimate += 3.75

        calorie_estimate = loop_through_ingredient_lists(sandwich["ingredients"],
                              restaurant,
                              checking_availability=False,
                              estimate_type="calories")

        # append its associated properties to the new sandwiches table object
        sandwich["restaurants"].append(restaurant_id)
        sandwich["costs"].append(cost_estimate)
        sandwich["calories2"].append(calorie_estimate)

  # print(f"Restaurants that sell the {sandwich['name']}:\t{sandwich['restaurants']}")

  # validate restaurant tagging
  if "default" not in sandwich["restaurants"]:
      print("Error, default not provided")

In [22]:
sandwiches_table.apply(update_sandwich, axis=1)

Error, default not provided
Error, default not provided
Possibly undervalued for 	'Reuben' for 	default, adding $3.75.
Possibly undervalued for 	'Rachel' for 	default, adding $3.75.
Possibly undervalued for 	'BLAT' for 	default, adding $3.75.
Possibly undervalued for 	'BLAT' for 	subway, adding $3.75.
Possibly undervalued for 	'BLT' for 	default, adding $3.75.
Possibly undervalued for 	'French Dip' for 	default, adding $3.75.
Possibly undervalued for 	'Garlic Roast Beef' for 	default, adding $3.75.
Possibly undervalued for 	'Roast Beef' for 	default, adding $3.75.
Possibly undervalued for 	'Elite Chicken & Bacon Ranch' for 	default, adding $3.75.
Possibly undervalued for 	'Honey Mustard BBQ Chicken' for 	default, adding $3.75.
Possibly undervalued for 	'Spicy Nacho Chicken' for 	default, adding $3.75.
Possibly undervalued for 	'Sweet Onion Chicken Teriyaki' for 	default, adding $3.75.
Possibly undervalued for 	'Teriyaki Blitz' for 	default, adding $3.75.
Possibly undervalued for 	'Uc B

,0
165,None
163,None
32,None
87,None
57,None
...,...
2,None
178,None
142,None
131,None


In [23]:
# clean up the restructured dataset
sandwiches_table.drop(columns=["cost", "calories", "restaurant"], inplace=True)
sandwiches_table.rename(columns={"calories2": "calories"}, inplace=True)

In [24]:
# add the _id fields back in
sandwiches_table.reset_index(drop=True, inplace=True)
sandwiches_table.reset_index(inplace=True)
ingredients_table.reset_index(inplace=True)

In [25]:
sandwiches_table.rename(columns={"index": "_id"}, inplace=True)
ingredients_table.rename(columns={"index": "_id"}, inplace=True)

In [26]:
sandwiches_table[:500]

,_id,cuisine,ingredients,rating,dietary_tags,reviews,name,restaurants,costs,calories
0,0,NaN,{'breads': {'artisan italian bread': ['artisan...,5,[],[],Pickleball Club,[subway],[6.99],[500]
1,1,NaN,{'breads': {'artisan italian bread': ['artisan...,1,[],[],The Monster,[subway],[10.99],[590]
2,2,jewish,"{'breads': {'bagel': ['bagel']}, 'meats': {}, ...",2,[vegetarian],[],Jewish Cream Cheese Sandwich,[default],[2.7],[345]
3,3,french,"{'breads': {'baguette': ['baguette']}, 'meats'...",1,[],[],French Beef Sandwich,[default],[5.0],[505]
4,4,british,"{'breads': {'baguette': ['baguette']}, 'meats'...",5,[],[],British Beef Sandwich,[default],[6.4],[640]
...,...,...,...,...,...,...,...,...,...,...
185,185,NaN,"{'breads': {}, 'meats': {}, 'cheeses': {'ricot...",4,[vegetarian],[],Ricotta Sandwich,[default],[3.2],[175]
186,186,NaN,"{'breads': {}, 'meats': {}, 'cheeses': {}, 've...",1,[vegan],[],Veggie Delite,"[subway, default]","[5.99, 6.75]","[220, 100]"
187,187,NaN,"{'breads': {}, 'meats': {}, 'cheeses': {}, 've...",1,[vegan],[],Truly Vegan,"[mr_pickles, default]","[13.99, 6]","[310, 310]"
188,188,NaN,"{'breads': {}, 'meats': {}, 'cheeses': {}, 've...",4,[vegan],[],Santa Maria Tri-Tip,"[mr_pickles, default]","[15.99, 5]","[250, 250]"


# Download the restructured dataset

In [31]:
def download_json(filename, data):
  # Convert the filtered DataFrame to JSON
  json_data = data.to_json(orient='records',
                                    indent=4,
                                    lines=True)
  json_data = "[\n" + json_data.replace("}\n\n    {", "},\n    {") + "\n]"

  # Save the JSON data to a file
  with open(filename, 'w') as f:
    f.write(json_data)

  # Download the file
  files.download(filename)

In [32]:
download_json("restaurant_ingredients.json", ingredients_table)
download_json("sandwiches.json", sandwiches_table)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>